In [1]:
# Import all pacakges needed for notebook
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#load data to a dataframe using an API call
api = KaggleApi()
api.authenticate()
api.dataset_download_file('thedevastator/cancer-patients-and-air-pollution-a-new-link','cancer patient data sets.csv', path='./')
df = pd.read_csv('cancer%20patient%20data%20sets.csv')
df

,index,Patient Id,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Level
0,0,P1,33,1,2,4,5,4,3,2,...,3,4,2,2,3,1,2,3,4,Low
1,1,P10,17,1,3,1,5,3,4,2,...,1,3,7,8,6,2,1,7,2,Medium
2,2,P100,35,1,4,5,6,5,5,4,...,8,7,9,2,1,4,6,7,2,High
3,3,P1000,37,1,7,7,7,7,6,7,...,4,2,3,1,4,5,6,7,5,High
4,4,P101,46,1,6,8,7,7,7,6,...,3,2,4,1,4,2,4,2,3,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,P995,44,1,6,7,7,7,7,6,...,5,3,2,7,8,2,4,5,3,High
996,996,P996,37,2,6,8,7,7,7,6,...,9,6,5,7,2,4,3,1,4,High
997,997,P997,25,2,4,5,6,5,5,4,...,8,7,9,2,1,4,6,7,2,High
998,998,P998,18,2,6,8,7,7,7,6,...,3,2,4,1,4,2,4,2,3,High


In [2]:
##Train Test Split
from sklearn.model_selection import train_test_split
df1 = df.drop(['index', 'Patient Id'], axis=1)
X = df.drop(['index','Level', 'Patient Id'], axis=1)
y = df['Level']

display(X.head())
display(y.head())

# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=104, test_size=0.25, shuffle=True)

,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,Obesity,...,Coughing of Blood,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring
0,33,1,2,4,5,4,3,2,2,4,...,4,3,4,2,2,3,1,2,3,4
1,17,1,3,1,5,3,4,2,2,2,...,3,1,3,7,8,6,2,1,7,2
2,35,1,4,5,6,5,5,4,6,7,...,8,8,7,9,2,1,4,6,7,2
3,37,1,7,7,7,7,6,7,7,7,...,8,4,2,3,1,4,5,6,7,5
4,46,1,6,8,7,7,7,6,7,7,...,9,3,2,4,1,4,2,4,2,3


0       Low
1    Medium
2      High
3      High
4      High
Name: Level, dtype: object

In [7]:
# Model creation and Backwards Feature Selection attempt

# Import the required libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFECV

# Create a random forest classifier model
rfc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# create the feature selector object
selector = RFECV(estimator=rfc, step=1, cv=5, min_features_to_select=10)

# fit the feature selector on the training data
selector = selector.fit(X_train, y_train)

# print the selected features
print("Selected Features:", X_train.columns[selector.support_])

# create a new random forest classifier with the selected features
rfc_selected = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
X_train_selected = X_train[X_train.columns[selector.support_]]
X_test_selected = X_test[X_train.columns[selector.support_]]
rfc_selected.fit(X_train_selected, y_train)

# predict on the testing data
y_train_pred = rfc_selected.predict(X_train_selected)
y_test_pred = rfc_selected.predict(X_test_selected)

# calculate the accuracy of the model on the testing data
accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", accuracy)
accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", accuracy)

Selected Features: Index(['Alcohol use', 'Dust Allergy', 'Balanced Diet', 'Obesity',
       'Passive Smoker', 'Coughing of Blood', 'Fatigue', 'Shortness of Breath',
       'Wheezing', 'Clubbing of Finger Nails'],
      dtype='object')
Train Accuracy: 0.992
Test Accuracy: 0.984


In [4]:
# Better, but still high!  We will continue to attemp different techniques to solve this issue.
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid=param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)

RandomForestClassifier(max_depth=6, max_features='sqrt', max_leaf_nodes=9)


In [8]:
# Create a random forest classifier model
rfc = RandomForestClassifier(max_depth=6, max_features='sqrt', max_leaf_nodes=9, random_state=42)

# create the feature selector object
selector = RFECV(estimator=rfc, step=1, cv=5, min_features_to_select=10)

# fit the feature selector on the training data
selector = selector.fit(X_train, y_train)

# print the selected features
print("Selected Features:", X_train.columns[selector.support_])

# create a new random forest classifier with the selected features
rfc_selected = RandomForestClassifier(max_depth=6, max_features='sqrt', max_leaf_nodes=9, random_state=42)
X_train_selected = X_train[X_train.columns[selector.support_]]
X_test_selected = X_test[X_train.columns[selector.support_]]
rfc_selected.fit(X_train_selected, y_train)

# predict on the testing data
y_train_pred = rfc_selected.predict(X_train_selected)
y_test_pred = rfc_selected.predict(X_test_selected)

# calculate the accuracy of the model on the testing data
accuracy = accuracy_score(y_train, y_train_pred)
print("Test Accuracy:", accuracy)
accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", accuracy)

Selected Features: Index(['Alcohol use', 'Dust Allergy', 'Balanced Diet', 'Obesity',
       'Passive Smoker', 'Coughing of Blood', 'Fatigue', 'Shortness of Breath',
       'Wheezing', 'Clubbing of Finger Nails'],
      dtype='object')
Test Accuracy: 1.0
Test Accuracy: 1.0


In [14]:
depth = [2,3,4,5]
est = [25,50,75,100]
for i in depth:
    for j in est:
        print('Max Depth: ', i, 'N Estimators: ',j)
        rfc = RandomForestClassifier(n_estimators=j, max_depth=i, random_state=42)
        selector = RFECV(estimator=rfc, step=1, cv=5, min_features_to_select=10)
        selector = selector.fit(X_train, y_train)
        #print("Selected Features:", X_train.columns[selector.support_])
        rfc_selected = RandomForestClassifier(n_estimators=j, max_depth=i, random_state=42)
        X_train_selected = X_train[X_train.columns[selector.support_]]
        X_test_selected = X_test[X_train.columns[selector.support_]]
        rfc_selected.fit(X_train_selected, y_train)
        y_train_pred = rfc_selected.predict(X_train_selected)
        y_test_pred = rfc_selected.predict(X_test_selected)
        accuracy = accuracy_score(y_train, y_train_pred)
        print("Train Accuracy:", accuracy)
        accuracy = accuracy_score(y_test, y_test_pred)
        print("Test Accuracy:", accuracy)

Max Depth:  2 N Estimators:  25
Train Accuracy: 0.9733333333333334
Test Accuracy: 0.96
Max Depth:  2 N Estimators:  50
Train Accuracy: 0.9333333333333333
Test Accuracy: 0.916
Max Depth:  2 N Estimators:  75
Train Accuracy: 0.9733333333333334
Test Accuracy: 0.96
Max Depth:  2 N Estimators:  100
Train Accuracy: 0.9733333333333334
Test Accuracy: 0.96
Max Depth:  3 N Estimators:  25
Train Accuracy: 0.992
Test Accuracy: 0.984
Max Depth:  3 N Estimators:  50
Train Accuracy: 0.9853333333333333
Test Accuracy: 0.964
Max Depth:  3 N Estimators:  75
Train Accuracy: 0.9933333333333333
Test Accuracy: 0.98
Max Depth:  3 N Estimators:  100
Train Accuracy: 0.9933333333333333
Test Accuracy: 0.98
Max Depth:  4 N Estimators:  25
Train Accuracy: 1.0
Test Accuracy: 1.0
Max Depth:  4 N Estimators:  50
Train Accuracy: 1.0
Test Accuracy: 1.0
Max Depth:  4 N Estimators:  75
Train Accuracy: 1.0
Test Accuracy: 1.0
Max Depth:  4 N Estimators:  100
Train Accuracy: 1.0
Test Accuracy: 1.0
Max Depth:  5 N Estimators:

In [15]:
i=25
j=3
print('Max Depth: ', i, 'N Estimators: ',j)
rfc = RandomForestClassifier(n_estimators=j, max_depth=i, random_state=42)
selector = RFECV(estimator=rfc, step=1, cv=5, min_features_to_select=10)
selector = selector.fit(X_train, y_train)
print("Selected Features:", X_train.columns[selector.support_])
rfc_selected = RandomForestClassifier(n_estimators=j, max_depth=i, random_state=42)
X_train_selected = X_train[X_train.columns[selector.support_]]
X_test_selected = X_test[X_train.columns[selector.support_]]
rfc_selected.fit(X_train_selected, y_train)
y_train_pred = rfc_selected.predict(X_train_selected)
y_test_pred = rfc_selected.predict(X_test_selected)
accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", accuracy)
accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", accuracy)

Max Depth:  25 N Estimators:  3
Selected Features: Index(['Alcohol use', 'OccuPational Hazards', 'Genetic Risk', 'Obesity',
       'Chest Pain', 'Coughing of Blood', 'Fatigue', 'Wheezing',
       'Swallowing Difficulty', 'Frequent Cold'],
      dtype='object')
Train Accuracy: 1.0
Test Accuracy: 1.0
